# Collision-Avoidance Velocity Control with Control Barrier Function

In [87]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

In [ ]:
class RaceTrack():
    def __init__(
            self,
            wheel_base: float = 2.5, # [m]
            max_vel_abs: float = 15.0, # [m/s]
            max_accel_abs: float = 3.0, # [m/s^2]
            ref_path: np.ndarray = np.array([[-100.0, 0.0], [100.0, 0.0]]),
            ref_inner_contour: np.ndarray = np.array([[-100.0, 0.0], [100.0, 0.0]]),
            ref_outer_contour: np.ndarray = np.array([[-100.0, 0.0], [100.0, 0.0]]),
            ref_vel: np.ndarray = np.array([10.0, 10.0]), # [m/s]
            delta_t: float = 0.05, # [s]
            cbf_1: callable = None,
            cbf_2: callable = None,
            cbf_safe_dist = None,
            visualize: bool = True,
        ) -> None:
        """initialize racetrack environment with two cars.
        state variables:
            x1: car1 location on the reference path [m]
            v1: car1 velocity along the reference path [m/s]
            x2: car2 location on the reference path [m]
            v2: car2 velocity along the reference path [m/s]
        control input:
            v1_input: car1 velocity input [m/s]
            v2_input: car2 velocity input [m/s]
        Note: dynamics of the cars are modeled as first-order delay system.
        """
        # vehicle parameters
        self.delay_time_constant = 0.2 # [s]
        self.max_vel_abs = max_vel_abs # [m/s]
        self.max_acc_abs = max_accel_abs # [m/s^2]
        self.delta_t = delta_t #[s]

        # control barrier function
        self.cbf_1 = cbf_1 # for car1
        self.cbf_2 = cbf_2 # for car2
        self.cbf_safe_dist = cbf_safe_dist # safety margin

        # visualization settings
        self.vehicle_w = 1.00 # vehicle width [m]
        self.vehicle_l = 1.25 # vehicle length [m]
        self.view_x_lim_min, self.view_x_lim_max =  -8.0, +38.0
        self.view_y_lim_min, self.view_y_lim_max =  +0.5, +27.0
        self.black_color = "#666666" # black
        self.safe_color  = "#005AFF" # green
        self.car1_color =  "#FF4B00" # orange
        self.car2_color =  "#005AFF" # blue
        self.miniview_x_lim_min, self.miniview_x_lim_max =  -35.0, +25.0
        self.miniview_y_lim_min, self.miniview_y_lim_max =  -17.0, +17.0

        # load reference path
        self.ref_path = ref_path
        self.ref_inner_contour = ref_inner_contour
        self.ref_outer_contour = ref_outer_contour
        self.ref_vel = ref_vel
        self.min_ref_vel, self.max_ref_vel = np.min(ref_vel), np.max(ref_vel)

        # calc path_length of ref_path for each row
        self.ref_path_length = np.zeros(len(ref_path))
        for i in range(1, len(ref_path)):
            self.ref_path_length[i] = self.ref_path_length[i-1] + np.linalg.norm(ref_path[i] - ref_path[i-1])

        # reset environment
        self.visualize_flag = visualize
        self.reset()

    def reset(
            self, 
            init_state: np.ndarray = np.array([0.0, 0.0, 0.0, 0.0]), # [x1, v1, x2, v2]
        ) -> None:
        """reset environment to initial state"""

        # reset state variables
        self.state = init_state

        # clear animation frames
        self.frames = []

        if self.visualize_flag:
            # prepare figure
            self.fig = plt.figure(figsize=(9,9))
            self.main_ax = plt.subplot2grid((3, 2), (0, 0), rowspan=2, colspan=2)
            self.car1_info_ax = plt.subplot2grid((3, 2), (2, 0))
            self.car2_info_ax = plt.subplot2grid((3, 2), (2, 1))

            # graph layout settings
            ## main view
            self.main_ax.set_aspect('equal')
            self.main_ax.set_xlim(self.view_x_lim_min, self.view_x_lim_max)
            self.main_ax.set_ylim(self.view_y_lim_min, self.view_y_lim_max)
            self.main_ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
            self.main_ax.tick_params(bottom=False, left=False, right=False, top=False)
            ## car 1 info view
            self.car1_info_ax.set_aspect('equal')
            self.car1_info_ax.set_xlim(self.miniview_x_lim_min, self.miniview_x_lim_max)
            self.car1_info_ax.set_ylim(self.miniview_y_lim_min, self.miniview_y_lim_max)
            self.car1_info_ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
            self.car1_info_ax.tick_params(bottom=False, left=False, right=False, top=False)
            ## car 2 info view
            self.car2_info_ax.set_aspect('equal')
            self.car2_info_ax.set_xlim(self.miniview_x_lim_min, self.miniview_x_lim_max)
            self.car2_info_ax.set_ylim(self.miniview_y_lim_min, self.miniview_y_lim_max)
            self.car2_info_ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
            self.car2_info_ax.tick_params(bottom=False, left=False, right=False, top=False)

            ## apply tight layout
            self.fig.tight_layout()
            ## add margin to the top
            self.fig.subplots_adjust(top=0.90, bottom=0.10, right=0.95, left=0.05)

            # add title
            self.fig.suptitle("\nCollision-Avoidance Velocity Control with Control Barrier Function", fontsize=16)

    def update(
            self, 
            u: np.ndarray, 
            delta_t: float = 0.0, 
            append_frame: bool = True, 
        ) -> None:
        """update state variables"""
        # keep previous states
        x1, v1, x2, v2 = self.state

        # prepare params
        dt = self.delta_t if delta_t == 0.0 else delta_t

        # limit control inputs
        v1_input = np.clip(u[0], -self.max_vel_abs, self.max_vel_abs)
        v2_input = np.clip(u[1], -self.max_vel_abs, self.max_vel_abs)

        # update state variables
        new_x1 = x1 + v1 * dt
        new_v1 = v1 + np.clip((v1_input - v1) / self.delay_time_constant, -self.max_acc_abs, self.max_acc_abs) * dt # apply acc limit
        new_x2 = x2 + v2 * dt
        new_v2 = v2 + np.clip((v2_input - v2) / self.delay_time_constant, -self.max_acc_abs, self.max_acc_abs) * dt # apply acc limit
        self.state = np.array([new_x1, new_v1, new_x2, new_v2])

        # record frame
        if append_frame:
            d_x1_to_x2, d_x2_to_x1 = self.get_distance_to_front_car()
            self.append_frame(v1_input=u[0], v2_input=u[1], d_x1_to_x2=d_x1_to_x2, d_x2_to_x1=d_x2_to_x1)

    def get_state(self) -> np.ndarray:
        """return state variables"""
        return self.state.copy()

    def get_referece_velocity(self) -> np.ndarray:
        """return reference velocity of car1 and car2"""
        x1, v1, x2, v2 = self.state
        car1_ref_vel = self.ref_vel[self.convert_x_to_path_index(x1)]
        car2_ref_vel = self.ref_vel[self.convert_x_to_path_index(x2)]
        return np.array([car1_ref_vel, car2_ref_vel]).flatten().copy()

    def convert_x_to_path_index(self, x: float) -> int:
        """return the index of the reference path according to the x position on the path"""
        path_index = 0
        for i in range(1, len(self.ref_path_length)):
            if math.fmod(x, self.ref_path_length[-1]) < self.ref_path_length[i-1]:
                path_index = i
                break
        return path_index

    def get_distance_to_front_car(self) -> float:
        """ get distance to the front car """
        # decide which car is in front
        x1, v1, x2, v2 = self.state
        path_index1 = self.convert_x_to_path_index(x1)
        path_index2 = self.convert_x_to_path_index(x2)
        if path_index1 < path_index2:
            # if car 2 is going ahead
            d_x1_to_x2 = self.ref_path_length[path_index2] - self.ref_path_length[path_index1]
            d_x2_to_x1 = self.ref_path_length[-1] - d_x1_to_x2
        else:
            # if car 1 is going ahead
            d_x2_to_x1 = self.ref_path_length[path_index1] - self.ref_path_length[path_index2]
            d_x1_to_x2 = self.ref_path_length[-1] - d_x2_to_x1
        return d_x1_to_x2, d_x2_to_x1

    def append_frame(self, v1_input: float, v2_input: float, d_x1_to_x2: float, d_x2_to_x1: float) -> None:
        """draw a frame of the animation."""
        # get current states
        x1, v1, x2, v2 = self.state
        # get car1 pose
        path_index = self.convert_x_to_path_index(x1)
        px1, py1 = self.ref_path[path_index]
        yaw1 = math.atan2(py1 - self.ref_path[path_index-1][1], px1 - self.ref_path[path_index-1][0])
        # get car2 pose
        path_index = self.convert_x_to_path_index(x2)
        px2, py2 = self.ref_path[path_index]
        yaw2 = math.atan2(py2 - self.ref_path[path_index-1][1], px2 - self.ref_path[path_index-1][0])

        ### main view ###
        # draw vehicle shapes
        vw, vl = self.vehicle_w, self.vehicle_l
        vehicle_shape_x = [-0.5*vl, -0.5*vl, +0.5*vl, +0.75*vl, +0.5*vl, -0.5*vl, -0.5*vl]
        vehicle_shape_y = [    0.0, +0.5*vw, +0.5*vw,      0.0, -0.5*vw, -0.5*vw,     0.0]
        ## draw car1
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw1, [px1, py1])
        frame = self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color=self.car1_color, linewidth=2.0, zorder=3)
        ## draw car2
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw2, [px2, py2])
        frame += self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color=self.car2_color, linewidth=2.0, zorder=3)

        # draw text info
        ## draw car1 velocity
        text = "car1 velocity = {v:>+6.1f} [m/s]".format(v=v1)
        frame += [self.main_ax.text(0.5, 0.07, text, ha='center', transform=self.main_ax.transAxes, fontsize=14, fontfamily='monospace')]
        ## draw car1 icon
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, np.pi/6.0, [4.3, 2.9]) # car icon on betagaki position
        frame += self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color=self.car1_color, linewidth=2.0, zorder=3)
        ## draw car2 velocity
        text = "car2 velocity = {v:>+6.1f} [m/s]".format(v=v2)
        frame += [self.main_ax.text(0.5, 0.03, text, ha='center', transform=self.main_ax.transAxes, fontsize=14, fontfamily='monospace')]
        ## draw car2 icon
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, np.pi/6.0, [4.3, 1.4]) # car icon on betagaki position
        frame += self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color=self.car2_color, linewidth=2.0, zorder=3)

        # draw the lane
        inner_contour_x = self.ref_inner_contour[:, 0]
        inner_contour_y = self.ref_inner_contour[:, 1]
        frame += self.main_ax.plot(inner_contour_x, inner_contour_y, color='#AAAAAA', linestyle="dashed", linewidth=1.0)
        outer_contour_x = self.ref_outer_contour[:, 0]
        outer_contour_y = self.ref_outer_contour[:, 1]
        frame += self.main_ax.plot(outer_contour_x, outer_contour_y, color='#AAAAAA', linestyle="dashed", linewidth=1.0)

        # visualize the reference velocity with color map
        color_list = []
        for i in range(0, len(self.ref_path)):
            # normalize target ref_vel with self.min_ref_vel and self.max_ref_vel
            color_list.append(plt.cm.coolwarm((self.ref_vel[i] - self.min_ref_vel) / (self.max_ref_vel - self.min_ref_vel)))
        frame += [self.main_ax.scatter(self.ref_path[:, 0], self.ref_path[:, 1], color=color_list, s=20, zorder=2, alpha=0.4)]

        # if cbf_1 nor cbf_2 is None, visualize car1 and car2 info
        if self.cbf_1 is not None and self.cbf_2 is not None and self.cbf_safe_dist is not None:
            # common
            pos_list = np.linspace(self.miniview_x_lim_min, self.miniview_x_lim_max, 300)
            dist_list = -pos_list
            car_shape_x = [0.0, 0.0, 2.0, 3.0, 3.0, 0.0]
            car_shape_y = [0.0, 2.2, 2.2, 1.3, 0.0, 0.0]

            ### car1 info view ###
            v1_cmd_list = [cbf_car1_vmax(d_x1_to_x2, self.cbf_safe_dist) for d_x1_to_x2 in dist_list]
            frame += self.car1_info_ax.plot(pos_list, v1_cmd_list, color=self.car1_color, label="vmax of car1")
            frame += [self.car1_info_ax.fill_between(pos_list, v1_cmd_list, self.miniview_y_lim_min, color=self.car1_color, alpha=0.2)]
            frame += [self.car1_info_ax.axvline(-self.cbf_safe_dist, color=self.black_color, linestyle='dashed', label="safety boundary")]
            point_car1 = patches.Circle([-d_x1_to_x2, v1_input], radius=0.3, fc=self.car1_color, ec=self.car1_color, linewidth=2.0, zorder=3)
            frame += [self.car1_info_ax.add_artist(point_car1)]
            # draw car2 shape
            car2_shape_x, car2_shape_y = \
                self._affine_transform(car_shape_x, car_shape_y, 0.0, [0.0, 1.5])
            frame += self.car1_info_ax.plot(car2_shape_x, car2_shape_y, color=self.car2_color, linewidth=2.0)
            wheel_rear  = patches.Circle([0.75, 0.7], radius=0.4, fc='white', ec=self.car2_color, linewidth=2.0)
            wheel_front = patches.Circle([2.25, 0.7], radius=0.4, fc='white', ec=self.car2_color, linewidth=2.0)
            frame += [self.car1_info_ax.add_artist(wheel_rear)]
            frame += [self.car1_info_ax.add_artist(wheel_front)]
            # draw car1 shape
            car1_shape_x, car1_shape_y = \
                self._affine_transform(car_shape_x, car_shape_y, 0.0, [-d_x1_to_x2, 1.5])
            frame += self.car1_info_ax.plot(car1_shape_x, car1_shape_y, color=self.car1_color, linewidth=2.0)
            wheel_rear  = patches.Circle([0.75-d_x1_to_x2, 0.7], radius=0.4, fc='white', ec=self.car1_color, linewidth=2.0)
            wheel_front = patches.Circle([2.25-d_x1_to_x2, 0.7], radius=0.4, fc='white', ec=self.car1_color, linewidth=2.0)
            frame += [self.car1_info_ax.add_artist(wheel_rear)]
            frame += [self.car1_info_ax.add_artist(wheel_front)]
            # add vertical axis arrow and text
            frame += [self.car1_info_ax.arrow(0.0, self.miniview_y_lim_min, 0.0, self.miniview_y_lim_max-self.miniview_y_lim_min-3.0, head_width=1.0, head_length=1.5, fc='black', ec='black')]
            varrow_text = "vel. cmd. \n of car1"
            frame += [self.car1_info_ax.text(0.68, 0.85, varrow_text, ha='left', transform=self.car1_info_ax.transAxes, fontsize=10, fontfamily='monospace')]
            # add horizontal axis arrow and text
            frame += [self.car1_info_ax.arrow(self.miniview_x_lim_min, 0.0, self.miniview_x_lim_max-self.miniview_x_lim_min-3.0, 0.0, head_width=1.0, head_length=1.5, fc='black', ec='black')]
            harrow_text = "relative\nposition"
            frame += [self.car1_info_ax.text(0.80, 0.30, harrow_text, ha='left', transform=self.car1_info_ax.transAxes, fontsize=10, fontfamily='monospace')]
            # add text "Safe Region"
            frame += [self.car1_info_ax.text(0.01, 0.05, "Safe Command Region", ha='left', transform=self.car1_info_ax.transAxes, fontsize=12, fontfamily='monospace', color=self.car1_color)]

            ### car2 info view ###
            v2_cmd_list = [cbf_car2_vmax(d_x2_to_x1, self.cbf_safe_dist) for d_x2_to_x1 in dist_list]
            frame += self.car2_info_ax.plot(pos_list, v2_cmd_list, color=self.car2_color, label="vmax of car2")
            frame += [self.car2_info_ax.fill_between(pos_list, v2_cmd_list, self.miniview_y_lim_min, color=self.car2_color, alpha=0.2)]
            frame += [self.car2_info_ax.axvline(-self.cbf_safe_dist, color=self.black_color, linestyle='dashed', label="safety boundary")]
            point_car2 = patches.Circle([-d_x2_to_x1, v2_input], radius=0.3, fc=self.car2_color, ec=self.car2_color, linewidth=2.0, zorder=3)
            frame += [self.car2_info_ax.add_artist(point_car2)]
            # draw car1 shape
            car2_shape_x, car2_shape_y = \
                self._affine_transform(car_shape_x, car_shape_y, 0.0, [0.0, 1.5])
            frame += self.car2_info_ax.plot(car2_shape_x, car2_shape_y, color=self.car1_color, linewidth=2.0)
            wheel_rear  = patches.Circle([0.75, 0.7], radius=0.4, fc='white', ec=self.car1_color, linewidth=2.0)
            wheel_front = patches.Circle([2.25, 0.7], radius=0.4, fc='white', ec=self.car1_color, linewidth=2.0)
            frame += [self.car2_info_ax.add_artist(wheel_rear)]
            frame += [self.car2_info_ax.add_artist(wheel_front)]
            # draw car2 shape
            car2_shape_x, car2_shape_y = \
                self._affine_transform(car_shape_x, car_shape_y, 0.0, [-d_x2_to_x1, 1.5])
            frame += self.car2_info_ax.plot(car2_shape_x, car2_shape_y, color=self.car2_color, linewidth=2.0)
            wheel_rear  = patches.Circle([0.75-d_x2_to_x1, 0.7], radius=0.4, fc='white', ec=self.car2_color, linewidth=2.0)
            wheel_front = patches.Circle([2.25-d_x2_to_x1, 0.7], radius=0.4, fc='white', ec=self.car2_color, linewidth=2.0)
            frame += [self.car2_info_ax.add_artist(wheel_rear)]
            frame += [self.car2_info_ax.add_artist(wheel_front)]
            # add vertical axis arrow and text
            frame += [self.car2_info_ax.arrow(0.0, self.miniview_y_lim_min, 0.0, self.miniview_y_lim_max-self.miniview_y_lim_min-3.0, head_width=1.0, head_length=1.5, fc='black', ec='black')]
            varrow_text = "vel. cmd. \n of car2"
            frame += [self.car2_info_ax.text(0.68, 0.85, varrow_text, ha='left', transform=self.car2_info_ax.transAxes, fontsize=10, fontfamily='monospace')]
            # add horizontal axis arrow and text
            frame += [self.car2_info_ax.arrow(self.miniview_x_lim_min, 0.0, self.miniview_x_lim_max-self.miniview_x_lim_min-3.0, 0.0, head_width=1.0, head_length=1.5, fc='black', ec='black')]
            harrow_text = "relative\nposition"
            frame += [self.car2_info_ax.text(0.80, 0.30, harrow_text, ha='left', transform=self.car2_info_ax.transAxes, fontsize=10, fontfamily='monospace')]
            # add text "Safe Region"
            frame += [self.car2_info_ax.text(0.01, 0.05, "Safe Command Region", ha='left', transform=self.car2_info_ax.transAxes, fontsize=12, fontfamily='monospace', color=self.car2_color)]

        # append frame
        self.frames.append(frame)

    # rotate shape and return location on the x-y plane.
    def _affine_transform(self, xlist: list, ylist: list, angle: float, translation: list=[0.0, 0.0]) -> list:
        transformed_x = []
        transformed_y = []
        if len(xlist) != len(ylist):
            print("[ERROR] xlist and ylist must have the same size.")
            raise AttributeError

        for i, xval in enumerate(xlist):
            transformed_x.append((xlist[i])*np.cos(angle)-(ylist[i])*np.sin(angle)+translation[0])
            transformed_y.append((xlist[i])*np.sin(angle)+(ylist[i])*np.cos(angle)+translation[1])
        transformed_x.append(transformed_x[0])
        transformed_y.append(transformed_y[0])
        return transformed_x, transformed_y

    def show_animation(self, interval_ms: int) -> None:
        """show animation of the recorded frames"""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval_ms) # blit=True
        html = display.HTML(ani.to_jshtml())
        display.display(html)
        plt.close()

    def save_animation(self, filename: str, interval: int, movie_writer: str="ffmpeg") -> None:
        """save animation of the recorded frames (ffmpeg required)"""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval)
        ani.save(filename, writer=movie_writer)

## [test simulation] ##

# simulation settings
sim_step = 10 # [steps]
delta_t = 0.10 # [s]

# load reference path and initialize racetrack environment
reference = np.genfromtxt('../data/f_shape_track.csv', delimiter=',', skip_header=1)
top_row   = np.genfromtxt('../data/f_shape_track.csv', delimiter=',', max_rows=1, dtype=str)
column_name = {col_name: i for i, col_name in enumerate(top_row)} # dict of {'column_name': column_number}
ref_path = np.vstack([
    [reference[:, column_name['opt_x']]], [reference[:, column_name['opt_y']]]
]).T
ref_inner_contour = np.vstack([
    [reference[:, column_name['inner_x']]], [reference[:, column_name['inner_y']]]
]).T
ref_outer_contour = np.vstack([
    [reference[:, column_name['outer_x']]], [reference[:, column_name['outer_y']]]
]).T
ref_vel = np.array([reference[:, column_name['ref_v']]]).T
racetrack = RaceTrack(
    ref_path=ref_path,
    ref_vel=ref_vel,
    ref_inner_contour=ref_inner_contour,
    ref_outer_contour=ref_outer_contour,
    visualize=True)
racetrack.reset(init_state=[20.0, 0.0, 5.0, 3.0]) # [x1, v1, x2, v2]

# run simulation
for i in range(sim_step):
    racetrack.update(u=[10.0, 10.0], delta_t=delta_t) # u is the control input to the vehicle, [ car1_vel, car2_vel ]

# show animation
racetrack.show_animation(interval_ms=delta_t*1000)

# save animation
# racetrack.save_animation("racetrack.mp4", interval=int(delta_t * 1000), movie_writer="ffmpeg") # ffmpeg is required to write mp4 file

In [ ]:
# IMPLEMENT LQR VELOCITY CONTROLLER HERE

In [ ]:
# define control barrier function for car1
def cbf_car1_vmax(d_x1_to_x2: float, dist_safe) -> float:
    # set params
    coef_car1_a = 1.0
    coef_car1_b = 5.0 #100.0

    # calc cbf
    v1_cmd_max = np.inf
    if d_x1_to_x2 > dist_safe:
        # safe
        v1_cmd_max = coef_car1_a * (d_x1_to_x2-dist_safe)
    else:
        # unsafe
        v1_cmd_max = coef_car1_b * (d_x1_to_x2-dist_safe)
    return v1_cmd_max
 
# define control barrier function for car2
def cbf_car2_vmax(d_x2_to_x1: float, dist_safe) -> float:
    # set params
    coef_car2_a = 0.2
    coef_car2_b = 5.0

    # calc cbf
    v2_cmd_max = np.inf
    if d_x2_to_x1 > dist_safe:
        # safe
        v2_cmd_max = coef_car2_a * (d_x2_to_x1-dist_safe)
    else:
        # unsafe
        v2_cmd_max = coef_car2_b * (d_x2_to_x1-dist_safe)
    return v2_cmd_max

# plot control barrier functions
dist_safe = 3.0
pos_list = np.linspace(-30.0, 15.0, 500)
dist_list = -pos_list
v1_cmd_list = [cbf_car1_vmax(d_x1_to_x2, dist_safe) for d_x1_to_x2 in dist_list]
v2_cmd_list = [cbf_car2_vmax(d_x2_to_x1, dist_safe) for d_x2_to_x1 in dist_list]
plt.plot(pos_list, v1_cmd_list, color='#FF4B00', label="cbf_car1")
plt.plot(pos_list, v2_cmd_list, color='#005AFF', label="cbf_car2")
plt.axvline(x=-dist_safe, color='black', linestyle='dashed', label="safety boundary")
plt.xlim(-30.0, 15.0)
plt.ylim(-20.0, 20.0)
plt.gca().set_aspect('equal', adjustable='box')
plt.plot([0.0, 3.0, 3.0, 0.0, 0.0], [0.0, 0.0, 2.0, 2.0, 0.0], color='black', label="front car") # plot car shape
plt.xlabel("position to the front car [m]")
plt.ylabel("velocity command [m/s]")
plt.title("Control Barrier Functions")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# control barrier function params
dist_safe = 3.0 # [m]

# simulation settings
sim_step = 400 # [steps]
delta_t = 0.10 # [s]
reference = np.genfromtxt('../data/f_shape_track.csv', delimiter=',', skip_header=1)
top_row   = np.genfromtxt('../data/f_shape_track.csv', delimiter=',', max_rows=1, dtype=str)
column_name = {col_name: i for i, col_name in enumerate(top_row)} # dict of {'column_name': column_number}
ref_path = np.vstack([
    [reference[:, column_name['opt_x']]], [reference[:, column_name['opt_y']]]
]).T
ref_inner_contour = np.vstack([
    [reference[:, column_name['inner_x']]], [reference[:, column_name['inner_y']]]
]).T
ref_outer_contour = np.vstack([
    [reference[:, column_name['outer_x']]], [reference[:, column_name['outer_y']]]
]).T
ref_vel = np.array([reference[:, column_name['ref_v']]]).T
racetrack = RaceTrack(
    ref_path=ref_path, 
    ref_vel=ref_vel, 
    ref_inner_contour=ref_inner_contour, 
    ref_outer_contour=ref_outer_contour, 
    cbf_1=cbf_car1_vmax,
    cbf_2=cbf_car2_vmax,
    cbf_safe_dist=dist_safe,
    visualize=True)
racetrack.reset(init_state=[5.0, 0.0, 12.5, 3.0]) # [x1, v1, x2, v2]

# scale reference velocity
car1_vel_scale = 2.0
car2_vel_scale = 1.0

# simulation loop
for i in range(sim_step):

    # [FOR TEST: CHANGE SPEED SCALE WITH TIME STEP]
    # if i < 150:
    #     car1_vel_scale = 2.0
    #     car2_vel_scale = 1.0
    # else:
    #     car1_vel_scale = 0.0
    #     car2_vel_scale = 10.0

    # get current states
    x1, v1, x2, v2 = racetrack.get_state()

    # calculate control input
    v1_cmd, v2_cmd = racetrack.get_referece_velocity()
    v1_cmd *= car1_vel_scale
    v2_cmd *= car2_vel_scale

    # limit commands with control barrier function to avoid collision
    d_x1_to_x2, d_x2_to_x1 = racetrack.get_distance_to_front_car()
    ## car 1
    v1_cmd = min(v1_cmd, cbf_car1_vmax(d_x1_to_x2, dist_safe))
    ## car 2
    v2_cmd = min(v2_cmd, cbf_car2_vmax(d_x2_to_x1, dist_safe))

    # update state variables
    racetrack.update(u=[v1_cmd, v2_cmd], delta_t=delta_t)

# show animation
racetrack.show_animation(interval_ms=delta_t*1000) # show animation
# save animation
racetrack.save_animation("cbf_vel_control.mp4", interval=int(delta_t * 1000), movie_writer="ffmpeg") # ffmpeg is required to write mp4 file